In [ ]:
    @staticmethod
    async def process_async_response(response: Any) -> Optional[Any]:
        """
        Process async API response with error handling.
        
        Args:
            response: Async API response
            
        Returns:
            Processed response or None
        """
        if response is None:
            logger.error("Failed to get async response after retries")
            return None
        
        # First, try to read the entire response content
        try:
            # Read the response content first
            content = await response.read()
            logger.debug(f"Successfully read response content, length: {len(content)}")
            
            # Then try to decode it as JSON
            try:
                return json.loads(content.decode('utf-8'))
            except json.JSONDecodeError as e:
                logger.error(f"Error decoding JSON from response content: {e}")
                # Return the raw text if JSON parsing fails
                return content.decode('utf-8', errors='replace')
                
        except aiohttp.ClientConnectionError as e:
            logger.error(f"Connection error while reading response: {e}")
            return {"error": "Connection closed while reading response"}
        except aiohttp.ClientResponseError as e:
            logger.error(f"Response error: {e.status} {e.message}")
            return {"error": f"HTTP error: {e.status} {e.message}"}
        except Exception as e:
            logger.error(f"Unexpected error processing async response: {e}", exc_info=True)
            return {"error": f"Failed to process response: {str(e)}"}
        
        return None